In [1]:
filename='virus dataset.fasta'

In [42]:
from Bio import SeqIO
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, Conv1D, MaxPooling1D,LSTM, Bidirectional,Dropout
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


In [3]:
seq_obj=SeqIO.parse(filename,'fasta')

In [4]:
sequences=[]

In [5]:
for seq in seq_obj:
    sequences.append(seq)

In [6]:
len(sequences)

34550

In [7]:
for record in sequences:
    seq_id=record.id
    seq_name=record.name
    seq_desc=record.description
    sequence=record.seq
    length=len(sequence)

In [8]:
seq_ids=[]
seq_lengths=[]
seqs=[]
seq_descs=[]
label=[]

In [9]:
for record in sequences:
    seq_id=record.id
    sequence=record.seq[200:2200]
    seq_desc=record.description
    if('coronavirus' in seq_desc):
        label.append('COVID-19')
    elif('Dengue' in seq_desc):
        label.append('Dengue')
    elif('Monkeypox' in seq_desc):
        label.append('Monkeypox')
    #elif('Hepatitis B' in seq_desc):
        #label.append('Hepatitis B')
    elif('Hepatitis C' in seq_desc):
        label.append('Hepatitis C')
    #elif('HIV' in seq_desc):
       # label.append('HIV')
    elif('Human respiratory syncytial' in seq_desc):
        label.append('Human orthopneumovirus')
    elif('TTV-like' in seq_desc):
        label.append('TTV-like')
    elif('Influenza A' in seq_desc):
        label.append('Influenza A')
    else:
        label.append(None)
    
    length=len(sequence)
    
    seq_ids.append(seq_id)
    seq_descs.append(seq_desc)
    
    seq_lengths.append(length)
    seqs.append(sequence)

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
dataframe=pd.DataFrame()
dataframe['seq_id']=seq_ids
dataframe['seq_desc']=seq_descs
dataframe['label']=label
dataframe['seq_length']=seq_lengths
dataframe['Dna Seq']=seqs

In [12]:
dataframe=dataframe[dataframe['seq_length'].notnull()]
dataframe=dataframe[dataframe['label'].notnull()]


In [13]:
dataframe.head(15)

,seq_id,seq_desc,label,seq_length,Dna Seq
10,ON675438.1,ON675438.1 Monkeypox virus isolate MPXV_USA_20...,Monkeypox,2000,"(C, G, A, A, G, A, G, A, G, A, A, A, G, A, G, ..."
11,JX878429.1,JX878429.1 Monkeypox virus isolate DRC 07-0662...,Monkeypox,2000,"(A, A, A, C, T, T, T, T, T, A, C, G, A, C, T, ..."
12,JX878428.1,JX878428.1 Monkeypox virus isolate DRC 07-0514...,Monkeypox,2000,"(A, A, A, C, T, T, T, T, T, A, C, G, A, C, T, ..."
13,JX878427.1,JX878427.1 Monkeypox virus isolate DRC 07-0480...,Monkeypox,2000,"(A, A, A, C, T, T, T, T, T, A, C, G, A, C, T, ..."
14,JX878426.1,JX878426.1 Monkeypox virus isolate DRC 07-0450...,Monkeypox,2000,"(G, T, T, A, G, T, C, A, A, G, A, T, A, T, T, ..."
15,JX878425.1,JX878425.1 Monkeypox virus isolate DRC 07-0354...,Monkeypox,2000,"(A, A, A, C, T, T, T, T, T, A, C, G, A, C, T, ..."
16,JX878424.1,JX878424.1 Monkeypox virus isolate DRC 07-0338...,Monkeypox,2000,"(A, A, A, C, T, T, T, T, T, A, C, G, A, C, T, ..."
17,JX878423.1,JX878423.1 Monkeypox virus isolate DRC 07-0337...,Monkeypox,2000,"(A, A, A, C, T, T, T, T, T, A, C, G, A, C, T, ..."
18,JX878422.1,JX878422.1 Monkeypox virus isolate DRC 07-0287...,Monkeypox,2000,"(A, A, A, C, T, T, T, T, T, A, C, G, A, C, T, ..."
19,JX878421.1,JX878421.1 Monkeypox virus isolate DRC 07-0286...,Monkeypox,2000,"(A, A, A, C, T, T, T, T, T, A, C, G, A, C, T, ..."


In [14]:
dataframe['label'].value_counts()

Dengue                    4333
TTV-like                  2487
Human orthopneumovirus    1724
Monkeypox                 1031
COVID-19                   875
Hepatitis C                811
Influenza A                545
Name: label, dtype: int64

In [15]:
dataframe=dataframe.drop(['seq_id','seq_length','seq_desc'],axis=1)

In [16]:
dataframe.tail(15)

,label,Dna Seq
32034,COVID-19,"(T, T, G, T, C, C, G, G, G, T, G, T, G, A, C, ..."
32035,COVID-19,"(T, T, G, T, C, C, G, G, G, T, G, T, G, A, C, ..."
32036,COVID-19,"(C, G, T, C, C, G, G, G, T, G, T, G, A, C, C, ..."
32068,COVID-19,"(G, T, C, C, G, T, G, T, T, G, C, A, G, C, C, ..."
32089,COVID-19,"(G, C, A, G, C, C, G, A, T, C, A, T, C, A, G, ..."
32090,COVID-19,"(T, G, C, A, G, C, C, G, A, T, C, A, T, C, A, ..."
32091,COVID-19,"(T, T, C, G, T, C, C, G, T, G, T, T, G, C, A, ..."
32092,COVID-19,"(C, G, T, C, C, G, T, G, T, T, G, C, A, G, C, ..."
33525,COVID-19,"(G, T, G, T, C, T, C, T, T, G, T, A, C, G, T, ..."
33588,COVID-19,"(T, G, T, C, G, A, A, G, A, T, C, A, A, C, A, ..."


In [17]:
def clean_Labels(x):
    
    if x=='Monkeypox':
        return 0
    #if x == 'HIV':
       # return 1
    if x=='Dengue':
        return 1
    if x == 'Human orthopneumovirus':
        return 2
    #if x=='Hepatitis B':
        #return 3
    if x == 'COVID-19':
        return 3
    if x=='Hepatitis C':
        return 4
    if x=='TTV-like':
        return 5
    if x=='Influenza A':
        return 6
    
    
    return float(x)

In [18]:
dataframe['label']=dataframe['label'].apply(clean_Labels)

In [19]:
dataframe['label'].to_numpy()

array([0, 0, 0, ..., 3, 3, 3], dtype=int64)

In [20]:
dataframe['label'].unique()

array([0, 1, 3, 2, 4, 5, 6], dtype=int64)

In [21]:
dataframe.head()

,label,Dna Seq
10,0,"(C, G, A, A, G, A, G, A, G, A, A, A, G, A, G, ..."
11,0,"(A, A, A, C, T, T, T, T, T, A, C, G, A, C, T, ..."
12,0,"(A, A, A, C, T, T, T, T, T, A, C, G, A, C, T, ..."
13,0,"(A, A, A, C, T, T, T, T, T, A, C, G, A, C, T, ..."
14,0,"(G, T, T, A, G, T, C, A, A, G, A, T, A, T, T, ..."


In [22]:
dataframe['Dna Seq']=dataframe['Dna Seq'].astype(str)

In [23]:
dataframe.head()

,label,Dna Seq
10,0,CGAAGAGAGAAAGAGATGGTTAGTCAAGATATTTTTCTTAGTACAA...
11,0,AAACTTTTTACGACTCCATCAGAAAGAGGTTTAATATTTTTGTGAG...
12,0,AAACTTTTTACGACTCCATCAGAAAGAGGTTTAATATTTTTGTGAG...
13,0,AAACTTTTTACGACTCCATCAGAAAGAGGTTTAATATTTTTGTGAG...
14,0,GTTAGTCAAGATATTTTTCTTAGTACAAAAGTCAATGTTTTAAAAT...


In [26]:
def Kmers_funct(seq, size=5):
    return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]

In [27]:
dataframe['words'] = dataframe.apply(lambda x: Kmers_funct(x['Dna Seq']), axis=1)
dataframe = dataframe.drop('Dna Seq', axis=1)

In [28]:
dataframe.head()

,label,words
10,0,"[cgaag, gaaga, aagag, agaga, gagag, agaga, gag..."
11,0,"[aaact, aactt, acttt, ctttt, ttttt, tttta, ttt..."
12,0,"[aaact, aactt, acttt, ctttt, ttttt, tttta, ttt..."
13,0,"[aaact, aactt, acttt, ctttt, ttttt, tttta, ttt..."
14,0,"[gttag, ttagt, tagtc, agtca, gtcaa, tcaag, caa..."


In [29]:
dna_texts = list(dataframe['words'])
for item in range(len(dna_texts)):
    dna_texts[item] =' '.join(dna_texts[item])

In [30]:
y_data = dataframe.iloc[:, 0].values

In [31]:
y_data

array([0, 0, 0, ..., 3, 3, 3], dtype=int64)

In [32]:
# Pre-processing of DNA sequences using CountVectorizer
vectorizer = CountVectorizer(analyzer='char', ngram_range=(4,4),max_features=768)
X = vectorizer.fit_transform(dna_texts)

In [33]:
X.shape

(11806, 768)

In [35]:
y_data.shape

(11806,)

In [36]:
y = keras.utils.to_categorical(y_data, num_classes=7)

In [37]:
y.shape

(11806, 7)

In [38]:
def create_model(input_shape, num_classes, type='CNN'):
    model = keras.Sequential()
    model.add(keras.layers.Reshape((input_shape[0], 1), input_shape=input_shape))
    model.add(keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(keras.layers.MaxPooling1D(pool_size=2))
    # bidirectional LSTM
    model.add(Bidirectional(LSTM(64, return_sequences = True)))
    # LSTM and droput
    model.add(LSTM(32, recurrent_dropout = 0.4))
    model.add(Dropout(0.2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dense(num_classes, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [39]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [40]:
# Convert sparse matrix to dense matrix
X_train = X_train.toarray()
X_test = X_test.toarray()

# Reshape the input to add the sequence length dimension
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [43]:
model = create_model(X_train.shape[1:], 7)

In [44]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 768, 1)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 766, 32)           128       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 383, 32)          0         
 1D)                                                             
                                                                 
 bidirectional_1 (Bidirectio  (None, 383, 128)         49664     
 nal)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 32)                20608     
                                                                 
 dropout (Dropout)           (None, 32)               

In [45]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
296/296 [==============================] - 123s 398ms/step - loss: 0.3331 - accuracy: 0.4535 - val_loss: 0.1922 - val_accuracy: 0.7172
Epoch 2/10
296/296 [==============================] - 114s 385ms/step - loss: 0.1169 - accuracy: 0.8629 - val_loss: 0.0402 - val_accuracy: 0.9763
Epoch 3/10
296/296 [==============================] - 122s 411ms/step - loss: 0.0466 - accuracy: 0.9644 - val_loss: 0.0201 - val_accuracy: 0.9809
Epoch 4/10
296/296 [==============================] - 119s 402ms/step - loss: 0.0873 - accuracy: 0.9100 - val_loss: 0.3485 - val_accuracy: 0.4513
Epoch 5/10
296/296 [==============================] - 119s 403ms/step - loss: 0.2779 - accuracy: 0.5413 - val_loss: 0.1353 - val_accuracy: 0.8768
Epoch 6/10
296/296 [==============================] - 124s 419ms/step - loss: 0.0857 - accuracy: 0.9099 - val_loss: 0.0352 - val_accuracy: 0.9627
Epoch 7/10
296/296 [==============================] - 125s 421ms/step - loss: 0.0427 - accuracy: 0.9592 - val_loss: 0.0347 -

In [46]:
# Evaluate the model on test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

74/74 [==============================] - 8s 104ms/step - loss: 0.0074 - accuracy: 0.9949
Test Loss: 0.007423311937600374
Test Accuracy: 0.9949195384979248


In [47]:
# Save the model
keras.models.save_model(model, 'model.h5')

In [48]:
model = keras.models.load_model('model.h5')

In [66]:
text = 'ATCTCACTTCCCCTCGTTCTCTTGCAGGACTTTGATTTTAACGAACTTAAATAAAAGCCCTGTTGTTTAGCGTATTGTTGCACTTGTCTGGTGGGATTGTGGCACTAATTTGCCTGCTCATCTAGGCAGTGGACATATGCTCAACACTGGGTATAATTCTAATTGAATACTATTTTTCAGTTAGAGCGTCGTGTCTCTTGTACGTCTCGGTCACAATACACGGTTTCGTCCGGTGCGTGGCAATTCGGGGCACATCATGTCTTTCGTGGCTGGTGTGATCGCGCAAGGTGCGCGCGGTACGTATCGAGCAGCGCTCAACTCTGAAAAACATCAAGACCATGTGTCTCTAACTGTGCCACTCTGTGGTTCAGGAAACCTGGTTGAAAAACTTTCACCATGGTTCATGGATGGCGAAAATGCCTATGAAGTGGTGAAGGCCATGTTACTTAAAAAAGAGCCACTTCTCTATGTGCCCATCCGGCTGGCTGGACACACTAGACACCTCCCAGGTCCTCGTGTGTACCTGGTTGAGAGGCTCATTGCTTGTGAAAATCCATTCATGGTTAACCAATTGGCTTATAGCTCTAGTGCAAATGGCAGCCTGGTTGGCACAACTTTGCAGGGCAAGCCTATTGGTATGTTCTTCCCTTATGACATCGAACTTGTCACAGGAAAGCAAAATATTCTCCTGCGCAAGTATGGCCGTGGTGGTTATCACTACACCCCATTCCACTATGAGCGAGACAACACCTCTTGCCCTGAGTGGATGGACGATTTTGAGGCGGATCCTAAAGGCAAATATGCCCAGAATCTGCTTAAGAAGTTGATTGGCGGTGATGTCACTCCAGTTGACCAATACATGTGTGGCGTTGATGGGAAACCCATTAGTGCCTACGCATTTTTAATGGCCAAGGATGGAATAACCAAACTGGCTGATGTTGAAGCGGACGTCGCAGCACGTGCTGATGACGAAGGCTTCATCACATTAAAGAACAATCTATATAGATTGGTTTGGCATGTTGAGCGTAAAGACGTTCCATATCCTAAGCAATCTATTTTTACTATTAATAGTGTGGTCCAAAAGGATGGTGTTGAAAACACTCCTCCTCACTATTTTACTCTTGGATGCAAAATTTTAACGCTCACCCCACGCAACAAGTGGAGTGGCGTTTCTGACTTGTCCCTCAAACAAAAACTCCTTTACACCTTCTATGGTAAGGAGTCACTTGAGAACCCAACCTACATTTACCACTCCGCATTCATTGAGTGTGGAAGTTGTGGTAATGATTCCTGGCTTACAGGGAATGCTATCCAAGGGTTTGCCTGTGGATGTGGGGCATCATATACAGCTAATGATGTCGAAGTCCAATCATCTGGCATGATTAAGCCAAATGCTCTTCTTTGTGCTACTTGCCCCTTTGCTAAGGGTGACAGCTGTTCTTCTAATTGCAAACATTCAGTTGCTCAGTTGGTTAGTTACCTTTCTGAACGCTGTAATGTTATTGCTGATTCTAAGTCCTTCACACTTATCTTTGGTGGCGTAGCTTACGCCTACTTTGGATGTGAGGAAGGTACTATGTACTTTGTGCCTAGAGCTAAGTCTGTTGTCTCAAGGATTGGAGACTCCATCTTTACAGGCTGTACTGGCTCTTGGAACAAGGTCACTCAAATTGCTAACATGTTCTTGGAACAGACTCAGCATTCCCTTAACTTTGTGGGAGAGTTCGTTGTCAACGATGTTGTCCTCGCAATTCTCTCTGGAACCACAACTAATGTTGACAAAATACGCCAGCTTCTCAAAGGTGTCACCATTGACAAGTTGCGTGATTATTTAGCTGACTATGACGTAGCAGTCACTGCCGGCCCATTCATGGATAATGCTATTAATGTTGGTGGTACAGGATTACAGTATGCCGCCATTACTGCACCTTATGTAGTTCTCACTGGCTTAGGTGAGTCCTTTAAGAAAGTTGCAACCATACCGTACAAGGTTTGCAACTCTGTTAAGGATACTCTGACTTATTATGCTCACAGCGTGTTGTACAGAGTTTTTCCTTATGACATGGATTCTGGTGTGTCATCCTTTAGTGAACTACTTTTTGATTGCGTTGATCTTTCAGTAGCTTCTACCTATTTTTTAGTCCGCCTCTTGCAAGATAAGACTGGCGACTTTATGTCTACAATTATTACTTCCTGCCAAACTGCTGTTAGTAAGCTTCTAGATACATGTTTTGAAGCTACAGAAGCAACATTTAACTTCTTGTTAGATTTGGCAGGATTGTTCAGAATCTTTCTTCGCAATGCCTATGTGTACACTTCACAAGGGTTTGTGGTGGTCAATGGCAAAGTTTCTACACTTGTCAAACAAGTGTTAGACTTGCTTAATAAGGGTATGCAACTTTTGCATACAAAGGTCTCCTGGGCTGGTTCTAATATCAGTGCTGTTATCTACAGCGGCAGGGAGTCTCTAATATTCCCATCGGGAACCTATTACTGTGTCACCACTAAGGCTAAGTCCGTTCAACAAGATCTTGACGTTATTTTGCCTGGTGAGTTTTCCAAGAAGCAGTTAGGACTGCTCCAACCTACTGACAATTCTACAACTGTTAGTGTTACTGTATCCAGTAACATGGTTGAAACTGTTGTGGGTCAACTTGAGCAAACTAATATGCATAGTCCTGATGTTATAGTAGGTGACTATGTCATTATTAGTGAAAAATTGTTTGTGCGTAGTAAGGAAGAAGACGGATTTGCCTTCTACCCTGCTTGCACTAATGGTCATGCTGTACCGACTCTCTTTAGACTTAAGGGAGGTGCACCTGTAAAAAAAGTAGCCTTTGGCGGTGATCAAGTACATGAGGTTGCTGCTGTAAGAAGTGTTACTGTCGAGTACAACATTCATGCTGTATTAGACACACTACTTGCTTCTTCTAGTCTTAGAACCTTTGTTGTAGATAAGTCTTTGTCAATTGAGGAGTTTGCTGACGTAGTAAAGGAACAAGTCTCAGACTTGCTTGTTAAATTACTGCGTGGAATGCCGATTCCAGATTTTGATTTAGACGATTTTATTGACGCACCATGCTATTGCTTTAACGCTGAGGGTGATGCATCTTGGTCTTCTACTATGATCTTCTCTCTTCACCCCGTCGAGTGTGACGAGGAGTGTTCTGAAGTAGAGGCTTCAGATTTAGAAGAAGGTGAATCAGAGTGCATTTCTGAGACTTCAACTGAACAAGTTGACGTTTCTCATGAGGTTTCTGACGACGAGTGGGCTGCTGCAGTTGATGAAG'

In [67]:
len(text)

3290

In [68]:
text=text[200:2000]

In [69]:
len(text)

1800

In [70]:
text = Kmers_funct(text, size=5)

In [71]:
text

['tacgt',
 'acgtc',
 'cgtct',
 'gtctc',
 'tctcg',
 'ctcgg',
 'tcggt',
 'cggtc',
 'ggtca',
 'gtcac',
 'tcaca',
 'cacaa',
 'acaat',
 'caata',
 'aatac',
 'ataca',
 'tacac',
 'acacg',
 'cacgg',
 'acggt',
 'cggtt',
 'ggttt',
 'gtttc',
 'tttcg',
 'ttcgt',
 'tcgtc',
 'cgtcc',
 'gtccg',
 'tccgg',
 'ccggt',
 'cggtg',
 'ggtgc',
 'gtgcg',
 'tgcgt',
 'gcgtg',
 'cgtgg',
 'gtggc',
 'tggca',
 'ggcaa',
 'gcaat',
 'caatt',
 'aattc',
 'attcg',
 'ttcgg',
 'tcggg',
 'cgggg',
 'ggggc',
 'gggca',
 'ggcac',
 'gcaca',
 'cacat',
 'acatc',
 'catca',
 'atcat',
 'tcatg',
 'catgt',
 'atgtc',
 'tgtct',
 'gtctt',
 'tcttt',
 'ctttc',
 'tttcg',
 'ttcgt',
 'tcgtg',
 'cgtgg',
 'gtggc',
 'tggct',
 'ggctg',
 'gctgg',
 'ctggt',
 'tggtg',
 'ggtgt',
 'gtgtg',
 'tgtga',
 'gtgat',
 'tgatc',
 'gatcg',
 'atcgc',
 'tcgcg',
 'cgcgc',
 'gcgca',
 'cgcaa',
 'gcaag',
 'caagg',
 'aaggt',
 'aggtg',
 'ggtgc',
 'gtgcg',
 'tgcgc',
 'gcgcg',
 'cgcgc',
 'gcgcg',
 'cgcgg',
 'gcggt',
 'cggta',
 'ggtac',
 'gtacg',
 'tacgt',
 'acgta',
 'cgtat',


In [72]:
joined_sentence = ' '.join(text)
joined_sentence

'tacgt acgtc cgtct gtctc tctcg ctcgg tcggt cggtc ggtca gtcac tcaca cacaa acaat caata aatac ataca tacac acacg cacgg acggt cggtt ggttt gtttc tttcg ttcgt tcgtc cgtcc gtccg tccgg ccggt cggtg ggtgc gtgcg tgcgt gcgtg cgtgg gtggc tggca ggcaa gcaat caatt aattc attcg ttcgg tcggg cgggg ggggc gggca ggcac gcaca cacat acatc catca atcat tcatg catgt atgtc tgtct gtctt tcttt ctttc tttcg ttcgt tcgtg cgtgg gtggc tggct ggctg gctgg ctggt tggtg ggtgt gtgtg tgtga gtgat tgatc gatcg atcgc tcgcg cgcgc gcgca cgcaa gcaag caagg aaggt aggtg ggtgc gtgcg tgcgc gcgcg cgcgc gcgcg cgcgg gcggt cggta ggtac gtacg tacgt acgta cgtat gtatc tatcg atcga tcgag cgagc gagca agcag gcagc cagcg agcgc gcgct cgctc gctca ctcaa tcaac caact aactc actct ctctg tctga ctgaa tgaaa gaaaa aaaaa aaaac aaaca aacat acatc catca atcaa tcaag caaga aagac agacc gacca accat ccatg catgt atgtg tgtgt gtgtc tgtct gtctc tctct ctcta tctaa ctaac taact aactg actgt ctgtg tgtgc gtgcc tgcca gccac ccact cactc actct ctctg tctgt ctgtg tgtgg gtggt tggtt ggttc gttca ttc

In [73]:
cv = CountVectorizer(ngram_range=(4,4),max_features=768)

In [74]:
V = cv.fit_transform([joined_sentence])

In [75]:
V.shape

(1, 768)

In [76]:
V=V.toarray()

In [77]:
V.shape

(1, 768)

In [78]:
V= np.reshape(V,(V.shape[0],V.shape[1],1))

In [79]:
V.shape

(1, 768, 1)

In [80]:
np.argmax(model.predict(V))

1/1 [==============================] - 0s 76ms/step


1